In [1]:
from google.cloud import bigquery
import pandas as pd

from google.oauth2 import service_account

key_path = "crunchbase-startups-b791b011b22c.json"

credentials = service_account.Credentials.from_service_account_file(
key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

sql = """
    SELECT *
    FROM `crunchbase-startups.startups.join_table`
"""

df = client.query(sql).to_dataframe()

In [2]:
df_op = df["status_operating"] == 1
df_ac = df["status_acquired"] == 1
df_ip = df["status_ipo"] == 1
df_cl = df["status_closed"] == 1

In [3]:
import numpy as np

In [4]:
category_encoded_op = np.zeros(len(df))
category_encoded_ac = np.zeros(len(df))
category_encoded_ip = np.zeros(len(df))
category_encoded_cl = np.zeros(len(df))
for c in set(np.array(df["category_code"])):
  if c == None:
    continue
  df_c = df["category_code"] == c
  stat_op = np.average(df["status_operating"][df_c]) 
  stat_ac = np.average(df["status_acquired"][df_c]) 
  stat_ip = np.average(df["status_ipo"][df_c]) 
  stat_cl = np.average(df["status_closed"][df_c]) 
  category_encoded_op[df_c], category_encoded_ac[df_c], \
  category_encoded_ip[df_c],category_encoded_cl[df_c], = stat_op, stat_ac, stat_ip, stat_cl
df["category_encoded_op"], df["category_encoded_ac"], \
df["category_encoded_ip"], df["category_encoded_cl"], = category_encoded_op, category_encoded_ac, category_encoded_ip, category_encoded_cl

In [5]:
country_encoded_op = np.zeros(len(df))
country_encoded_ac = np.zeros(len(df))
country_encoded_ip = np.zeros(len(df))
country_encoded_cl = np.zeros(len(df))
for c in set(np.array(df["country_code"])):
  if c == None:
    continue
  df_c = df["country_code"] == c
  stat_op = np.average(df["status_operating"][df_c]) 
  stat_ac = np.average(df["status_acquired"][df_c]) 
  stat_ip = np.average(df["status_ipo"][df_c]) 
  stat_cl = np.average(df["status_closed"][df_c]) 
  country_encoded_op[df_c], country_encoded_ac[df_c], \
  country_encoded_ip[df_c],country_encoded_cl[df_c], = stat_op, stat_ac, stat_ip, stat_cl
df["country_encoded_op"], df["country_encoded_ac"], \
df["country_encoded_ip"], df["country_encoded_cl"], = country_encoded_op, country_encoded_ac, country_encoded_ip, country_encoded_cl

In [6]:
state_encoded_op = np.zeros(len(df))
state_encoded_ac = np.zeros(len(df))
state_encoded_ip = np.zeros(len(df))
state_encoded_cl = np.zeros(len(df))
for c in set(np.array(df["state_code"])):
  if c == None:
    continue
  df_c = df["state_code"] == c
  stat_op = np.average(df["status_operating"][df_c]) 
  stat_ac = np.average(df["status_acquired"][df_c]) 
  stat_ip = np.average(df["status_ipo"][df_c]) 
  stat_cl = np.average(df["status_closed"][df_c]) 
  state_encoded_op[df_c], state_encoded_ac[df_c], \
  state_encoded_ip[df_c],state_encoded_cl[df_c], = stat_op, stat_ac, stat_ip, stat_cl
df["state_encoded_op"], df["state_encoded_ac"], \
df["state_encoded_ip"], df["state_encoded_cl"], = state_encoded_op, state_encoded_ac, state_encoded_ip, state_encoded_cl

In [7]:
project = client.project
dataset_ref = bigquery.DatasetReference(project, 'startups')

In [8]:
table_id = "join_table"
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)
print("Table {} contains {} columns.".format(table_id, len(table.schema)))

Table join_table contains 35 columns.


In [9]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
# job_config.schema_update_options = [
#     bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
# ]

In [10]:
job_config.schema =  table.schema + [
    bigquery.SchemaField("category_encoded_op", "FLOAT", mode="NULLABLE"), 
    bigquery.SchemaField("category_encoded_ac", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("category_encoded_ip", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("category_encoded_cl", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("country_encoded_op", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("country_encoded_ac", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("country_encoded_ip", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("country_encoded_cl", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("state_encoded_op", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("state_encoded_ac", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("state_encoded_ip", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("state_encoded_cl", "FLOAT", mode="NULLABLE")
    ]

In [11]:
job = client.load_table_from_dataframe(
    df, "crunchbase-startups.startups.join_table", job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

LoadJob<project=crunchbase-startups, location=US, id=671765d0-8ae0-449d-896f-5d35a186c0c1>